# Utilizando o BERTimbau para criação dos vetores

https://huggingface.co/neuralmind/bert-base-portuguese-cased

In [18]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import psycopg2 as pgsql
import pandas as pd

from transformers import AutoTokenizer  
from transformers import AutoModel
import torch

In [19]:
load_dotenv()

True

In [20]:
user=os.environ['user']
password=os.environ['password']
host=os.environ['host']
port=os.environ['port']
database=os.environ['database']

url = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
con = create_engine(url)

In [28]:
# Selecionando todos os campos textuais que serão utilizados para construir o vetor do produto

query_infos = """
    SELECT
        f.medicamento_id,
        CONCAT(clf.produto, ' ', clf.principio_ativo, ' ', 
            clf.tipo, ' ', clf.categoria, ' ', 
            clf.classe_terapeutica, ' ', clf.especialidade, ' ', 
            clf.fabricante) as descricao
    FROM fat_produto f
    INNER JOIN dim_classificacao_produto clf
        on f.medicamento_id = clf.medicamento_id
    WHERE
        1=1
        and f.deleted_at is null
    LIMIT 10;
"""

In [29]:
df = pd.read_sql(query_infos, con)

In [30]:
df

,medicamento_id,descricao
0,1,A Curitybina Ácido Salicílico Similar Calos E ...
1,2,A Saúde da Mulher Plumeria Lancifolia + Salici...
2,3,Advantage Max3 para Cães Imidacloprida + Perme...
3,4,Timosan Maleato De Timolol Similar Doenças Dos...
4,5,Timosopt Cloridrato De Dorzolamida + Maleato D...
5,6,Tintura de Allium sativum FSL Farma Associaçõe...
6,7,Tintura de Amora Negra Miligrama Homeopático O...
7,8,Tintura de Assa Peixe FSL Farma Associações Fi...
8,9,Tintura de Cana do Brejo FSL Farma Associações...
9,10,Tintura de Carqueja FSL Farma Associações Fito...


In [42]:
# função de vetorização

def query(text):
    tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

    model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
    input_ids = tokenizer.encode(f'{text}', return_tensors='pt')

    with torch.no_grad():
        outs = model(input_ids)
        encoded = outs[0][0, 1:-1]  # Ignore [CLS] and [SEP] special tokens
    vetor = encoded.mean(dim=0).detach().numpy()
    
    return list(vetor)

In [43]:
vector_list = []

for i in range(0, len(df)):
    output = query(df['descricao'][i])
    vector_list.append(output)

df['vetor'] = vector_list

/home/gustavo-cunha/Documentos/GitHub/catalogo_medicamentos/env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [44]:
df

,medicamento_id,descricao,vetor
0,1,A Curitybina Ácido Salicílico Similar Calos E ...,"[-0.009731463, -0.2187955, 0.24772458, -0.0101..."
1,2,A Saúde da Mulher Plumeria Lancifolia + Salici...,"[-0.08012641, -0.25941694, 0.6892007, -0.18553..."
2,3,Advantage Max3 para Cães Imidacloprida + Perme...,"[0.14458345, -0.06095767, 0.5265002, 0.1192908..."
3,4,Timosan Maleato De Timolol Similar Doenças Dos...,"[-0.019600984, -0.13994999, 0.29558375, -0.280..."
4,5,Timosopt Cloridrato De Dorzolamida + Maleato D...,"[0.15316992, -0.091649935, 0.29392785, -0.0913..."
5,6,Tintura de Allium sativum FSL Farma Associaçõe...,"[-0.0117131295, -0.19581011, 0.52420866, 0.058..."
6,7,Tintura de Amora Negra Miligrama Homeopático O...,"[0.11876611, -0.06176117, 0.3779615, -0.037188..."
7,8,Tintura de Assa Peixe FSL Farma Associações Fi...,"[0.035189904, -0.1919168, 0.543696, 0.10637081..."
8,9,Tintura de Cana do Brejo FSL Farma Associações...,"[-0.019185629, -0.2048706, 0.5076293, 0.174192..."
9,10,Tintura de Carqueja FSL Farma Associações Fito...,"[-0.0179521, -0.17272326, 0.50445193, 0.149744..."


In [46]:
len(df['vetor'][0]) # dimensão do vetor

768

### Criando tabela de vetores e populando com o teste

In [47]:
def exec_query(sql):
    conn = pgsql.connect(database = database, 
                     user = user, 
                     host= host,
                     password = password,
                     port = port)
    
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()
    cur.close()
    conn.close()

In [38]:
# criando a tabela de vetores
sql_create = """CREATE TABLE dim_vetores_teste (
            medicamento_id bigint PRIMARY KEY, 
            vetor vector(768)
            );
            """

exec_query(sql_create)

In [50]:
# populando a tabela de vetores

for n in range(0, len(df)):
    sql_insert = f""" 
    INSERT INTO dim_vetores_teste(medicamento_id, vetor) VALUES({df['medicamento_id'][n]}, '{df['vetor'][n]}');
    """
    exec_query(sql_insert)

### Buscando por proximidade simulando um campo de busca

In [51]:
df['descricao']

0    A Curitybina Ácido Salicílico Similar Calos E ...
1    A Saúde da Mulher Plumeria Lancifolia + Salici...
2    Advantage Max3 para Cães Imidacloprida + Perme...
3    Timosan Maleato De Timolol Similar Doenças Dos...
4    Timosopt Cloridrato De Dorzolamida + Maleato D...
5    Tintura de Allium sativum FSL Farma Associaçõe...
6    Tintura de Amora Negra Miligrama Homeopático O...
7    Tintura de Assa Peixe FSL Farma Associações Fi...
8    Tintura de Cana do Brejo FSL Farma Associações...
9    Tintura de Carqueja FSL Farma Associações Fito...
Name: descricao, dtype: object

In [53]:
vetor = query("medicamento para caes") # formulando um texto de busca que faça sentido com as descricoes para visualizar as opções mais proximas

query_find = f"SELECT medicamento_id FROM dim_vetores_teste ORDER BY vetor <=> '{vetor}' LIMIT 2;"

In [54]:
conn = pgsql.connect(database = database, 
                    user = user, 
                    host= host,
                    password = password,
                    port = port)

cur = conn.cursor()
cur.execute(query_find)
rows = cur.fetchall()
conn.commit()
conn.close()
for row in rows:
    print(row)

(3,)
(7,)
